# Price indifferent liquidity
This notebook lays out an idea how to deal with liquidity pools.

## Basic trading

First, we define a basic pool:

In [1]:
class Pool(object):
    def __init__(self, size):
        self.size = size

We create a trade pair covering two stables. We use UST and DAI because those two have the very nice property that they are both three-letters acronyms.

In [2]:
ust = Pool(1000)
dai = Pool(1000)

We define a function that calculates how much one is to receive given pool sizes and offer amount.

In [3]:
# askBaseAmount = askPool - cp / (offerPool + offerBaseAmount)
def calc_to_receive(pool1, pool2, amount):
    cp = pool1.size * pool2.size
    return pool2.size - cp / (pool1.size + amount)

In [4]:
print(calc_to_receive(ust, dai, 1))
print(calc_to_receive(ust, dai, 10))
print(calc_to_receive(ust, dai, 100))
print(calc_to_receive(ust, dai, 1000))

0.9990009990010549
9.900990099009846
90.90909090909088
500.0


We see that the more from `pool2` is taken, the less one is receiving.

Next, we create a function that executes a trade.

In [5]:
def do_trade(pool1, pool2, amount):
    to_receive = calc_to_receive(pool1, pool2, amount)
    pool1.size += amount
    pool2.size -= to_receive
    return to_receive

Max promised his best friend to give him 1000 DAI for this. Since he doesn't know how trading works, the does 10 trades where he buys sells 100 UST each time:

In [6]:
one_trade = calc_to_receive(ust, dai, 500)

five_trades = 0
for _ in range(5):
    five_trades += do_trade(ust, dai, 100)
    
print(one_trade, five_trades)

333.33333333333337 333.33333333333326


With each trade from UST to DAI, the he receives less DAI. Why? Because liquidity is added to the UST pool and taken away the DAI pool. Meaning, UST is losing value while DAI is gaining value.

In [7]:
print(ust.size)
print(dai.size)

1500
666.6666666666667


Now, if one would swap 1 DAI to UST, one would receive this much UST:

In [8]:
print(calc_to_receive(dai, ust, 1))
print(calc_to_receive(dai, ust, 10))
print(calc_to_receive(dai, ust, 100))

2.2466300549176594
22.167487684729167
195.6521739130435


Now Alex is thinking: Wow, I will add all 500 DAI I have and buy UST with that. Let's simulate what happens.

In [9]:
dai.size += 500
print(calc_to_receive(dai, ust, 1))
print(calc_to_receive(dai, ust, 10))
print(calc_to_receive(dai, ust, 100))

1.2846131886954026
12.747875354107691
118.42105263157896


Before adding liquidty, each UST was worth around 2.2 DAI. After adding new liquidity and buying DAI, Alex received much less than expected. By adding his liquidity, he lowered the value of UST.

How could we solve this problem? Meaning, how can we make it such that people can add liquidity without changing the price?

## Actual liquidity and virtual liquidity

To solve Alex's proble, Max had the idea to change liquidity adding such that adding liquidity affect the value of the trading pair less. This is done by splitting the pools into actual and virtual liquidity:

In [10]:
class Pool(object):
    def __init__(self, size):
        self.actual = size
        self.virtual = 0
        
    @property
    def size(self):
        return self.actual + self.virtual

    
ust = Pool(1000)
dai = Pool(1000)

def do_trade(pool1, pool2, amount):
    to_receive = calc_to_receive(pool1, pool2, amount)
    pool1.actual += amount
    pool2.actual -= to_receive
    return to_receive

Now, when adding actual liquidity to DAI, we also add virtual liquidity to UST to balance the pools.

In [11]:
def add_liquidity(pool1, pool2, amount):
    growth_factor = (pool1.actual + amount) / pool1.actual
    pool1.actual += amount
    pool2.virtual = (pool2.size * growth_factor) - pool2.size

In [12]:
one_trade = calc_to_receive(ust, dai, 500)

five_trades = 0
for _ in range(5):
    five_trades += do_trade(ust, dai, 100)
    
print(one_trade, five_trades)

333.33333333333337 333.33333333333326


In [13]:
add_liquidity(dai, ust, 500)
print('UST, actual amount: %.2f, virtual amount: %.2f' % (ust.actual, ust.virtual))
print('DAI, actual amount: %.2f, virtual amount: %.2f' % (dai.actual, dai.virtual))

UST, actual amount: 1500.00, virtual amount: 1125.00
DAI, actual amount: 1166.67, virtual amount: 0.00


We added liquidity to DAI. To keep the pools balanced, we also added virtual liquidity to UST.

In [14]:
print(calc_to_receive(dai, ust, 1))
print(calc_to_receive(dai, ust, 10))
print(calc_to_receive(dai, ust, 100))

2.248073080217182
22.308781869688573
207.23684210526335


We see that the price of UST/DAI is kept the same for trading 1 unit. When trading larger amount, the slippage (?) is smaller because the overall liquidity is higher now.

## Multiple currencies

The previous examples covered two stablecoins and trading between them. Next, we want to look at trading and changing liquidity when having more two currencies that are not stable, for example UST and BTC. We want to do this by introducing one currency with is used as a centerpoint. This means, all trades are done by first converting to the center currency, CTR, and then to the new currency.

We initialize the pools by assuming 20_000 UST = 1 BTC = 1000 CTR. (The CTR value is chosen randomly)

In [15]:
ust = Pool(20_000)
ctr = Pool(1_000)
btc = Pool(1)

In [16]:
def calc_to_receive_using_ctr(pool1, pool2, amount):
    to_receive = calc_to_receive(pool1, ctr, amount)
    to_receive = calc_to_receive(ctr, pool2, to_receive)
    return to_receive

In [17]:
calc_to_receive(ust, btc, 1)

4.99975001250208e-05

We see, 1 UST ~ 0.0005 BTC, which is to be expected.

In [18]:
def do_trade_using_ctr(pool1, pool2, amount):
    to_receive = do_trade(pool1, ctr, amount)
    to_receive = do_trade(ctr, pool2, to_receive)
    return to_receive

In [19]:
for _ in range(10):
    to_receive = do_trade_using_ctr(ust, btc, 100)
    print('%.5f\t%.1f' % (to_receive, ctr.size))

0.00498	1000.0
0.00493	1000.0
0.00488	1000.0
0.00483	1000.0
0.00478	1000.0
0.00474	1000.0
0.00469	1000.0
0.00465	1000.0
0.00460	1000.0
0.00456	1000.0


When selling UST for BTC, the amount of BTC to receive is smaller after each trade. The pool size of the centerpoint is staying the same, even if every trade is routed via this currency.

In [20]:
print(calc_to_receive_using_ctr(ust, btc, 1000))
_ = do_trade(ust, ctr, 1000)
print(calc_to_receive_using_ctr(ust, btc, 1000))

0.04140786749482406
0.03968253968253954
